# Settings all imports

In [3]:
import tqdm
import requests
import urllib.request

import numpy as np
import pandas as pd

from datetime import date, timedelta
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from bs4.element import Comment
# from google.colab import drive

# Create a web driver based on operating system. Please change the value of the `OPERATING_SYSTEM` constant to run this notebook on your system

In [28]:
OPERATING_SYSTEM = "mac"

if OPERATING_SYSTEM == "windows":
    executable_path = "drivers/windows/chromedriver.exe"
elif OPERATING_SYSTEM == "mac":
    executable_path = "drivers/mac/chromedriver"
elif OPERATING_SYSTEM == "linux":
    executable_path = "chromedriver"

base_url = 'https://www.jobs.bg/'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(executable_path=executable_path, options=chrome_options)


"""url = 'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging'
r = driver.get(url)
html = driver.execute_script("return document.documentElement.outerHTML")
soup = BeautifulSoup(html, 'html.parser')"""

'url = \'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging\'\nr = driver.get(url)\nhtml = driver.execute_script("return document.documentElement.outerHTML")\nsoup = BeautifulSoup(html, \'html.parser\')'

# Extracting the category names and ids so we can perform filters based on category

In [41]:
def extract_categories_dataframe():
    """"""
    url = "https://www.jobs.bg/index.php"
    r = driver.get(url)
    categories_button = driver.find_element_by_id("categoriesChip")
    categories_button.click()

    html = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(html, 'html.parser')

    categories_elem = soup.find('div',attrs={'id':'categoriesSelectSheet'})
    categories_elements = categories_elem.find_all("div", attrs={"class": "mdc-chip mdc-ripple-upgraded"})

    categories_info = []
    categories_info.append((56))

    for category_element in categories_elements:
        category_id = category_element['id'].split("_")[-1]
        categories_info.append(category_id)

    categories_info = pd.DataFrame(categories_info, columns=["category_id"])

    return categories_info

In [42]:
categories_info = extract_categories_dataframe()
categories_info

,category_id
0,56
1,36
2,45
3,57
4,6
5,38
6,3
7,29
8,52
9,17


In [38]:
categories_info = extract_categories_dataframe()
categories_info

,category_id,category_name
0,56,IT JOBS
1,36,"Ресторанти, Заведения, Хотели, Туризъм"
2,45,Търговия и Продажби
3,57,Производство
4,6,"Шофьори, Куриери, Транспорт, Логистика"
5,38,"Административни, Офис и Бизнес дейности"
6,3,"Архитектура, Строителство"
7,29,Инженери и Техници
8,52,Физически/Ръчен труд
9,17,Центрове за обслужване на клиенти и бизнес усл...
